# Module 1: Exploratory Data Analysis (EDA) and Validation

**Training Objective:** Master exploratory data analysis (EDA) techniques and data quality validation before starting the ML process.

**Scope:**
- Generating synthetic data with defects (outliers, missing values)
- Data profiling: schema, descriptive statistics
- Visualizations: histograms, box plots
- Outlier detection: IQR (Interquartile Range) method

## Context and Requirements

- **Training day:** Day 1 - Data Preparation Fundamentals
- **Notebook type:** Demo
- **Technical requirements:**
  - Databricks Runtime 14.x LTS or newer
  - Unity Catalog enabled
  - Permissions: CREATE TABLE, SELECT, MODIFY
- **Dependencies:** None (first notebook)
- **Execution time:** ~30 minutes

> **Note:** This notebook generates synthetic data - each run creates fresh data.

## Theoretical Introduction

**Why is EDA crucial in ML?**

Exploratory Data Analysis (EDA) is the first and most important step in any ML project. The **"Garbage In, Garbage Out"** principle means that even the best model won't help if the data quality is poor.

**What does EDA give us?**

| Aspect | Question | Consequence for ML |
|--------|----------|-------------------|
| **Data Quality** | Missing values? Duplicates? Impossible values? | Requires imputation or removal |
| **Distribution** | Normal or skewed data? | Affects model choice and scaling |
| **Outliers** | Extreme values? | Can "break" linear models |
| **Correlations** | Which features are correlated? | Helps with feature selection |

**IQR Method for Outliers:**

We use the **Interquartile Range (IQR)** because it is robust against outliers themselves (unlike standard deviation).

Rule: A point is an outlier if it is above $Q3 + 1.5 \times IQR$ or below $Q1 - 1.5 \times IQR$.

## Per-User Isolation

Run the initialization script for per-user catalog and schema isolation:

In [0]:
%run ./00_Setup

## Section 1: Data Generation (Bronze Layer)

We will generate a dataset representing raw customer data. We will intentionally introduce:
- **Outliers**: Extremely high salaries.
- **Missing Values**: Nulls in Age and Email.
- **Noise**: Inconsistent categories.

In [0]:
# Install Faker for synthetic data generation
%pip install faker

In [0]:
from faker import Faker
import random
from pyspark.sql.types import *

fake = Faker()
random.seed(42)

# Configuration
n_rows = 2000
data = []

for i in range(n_rows):
    # Simulate Outliers in Salary (1% of data)
    if random.random() < 0.01:
        salary = random.randint(500000, 1000000) # Outlier
    else:
        salary = random.randint(30000, 120000)   # Normal
        
    # Simulate Missing Age (5% of data)
    age = None if random.random() < 0.05 else random.randint(18, 85)
    
    row = (
        i, 
        fake.name(),
        age,
        salary,
        fake.date_between(start_date="-2y", end_date="today").strftime("%Y-%m-%d"),
        random.choice(["CRM", "WEB", "MOBILE", "PARTNER"]),
        random.choice(["USA", "UK", "DE", "FR", "PL"])
    )
    data.append(row)

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", IntegerType(), True),
    StructField("registration_date", StringType(), True),
    StructField("source", StringType(), True),
    StructField("country", StringType(), True)
])

df_raw = spark.createDataFrame(data, schema)
df_raw.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.customer_bronze")

print("✅ Generated 'customer_bronze' table.")

## Section 2: Data Profiling & Summary

**Why do we need EDA?**
Before training any model, we must understand our data. "Garbage In, Garbage Out" is the golden rule of ML.
EDA helps us answer:
1.  **Data Quality:** Are there missing values? Duplicates? Impossible values (e.g., Age = -5)?
2.  **Data Distribution:** Is the data normal or skewed? (Affects model choice).
3.  **Correlations:** Which features might be useful for prediction?

The first step is to look at the raw numbers.

In [0]:
# Load data
df = spark.table("customer_bronze")

# 1. Basic Display - Use the 'Data Profile' tab in the output below!
display(df)

In [0]:
dbutils.data.summarize(df)

Data Profiling: Display Customer Bronze Table 
This cell loads the 'customer_bronze' table and displays its contents for initial data profiling.

In [0]:
display(df.describe())

In [0]:
display(df.summary())

### Example 2.1: Statistical Summary
The `summary()` command provides count, mean, stddev, min, max, and quartiles.

In [0]:
# Check statistics for numerical columns
display(df.select("age", "salary").summary())

In [0]:
# Check Skewness
# Skewness > 1 indicates highly skewed data (long tail). 
# This suggests we might need Log Transformation later.
from pyspark.sql.functions import skewness

display(df.select(skewness("age").alias("age_skewness"), skewness("salary").alias("salary_skewness")))


## Section 3: Visualizations

We can use the built-in plotting tool in `display()` to visualize distributions.

**Task:**
1.  Click the **+** icon in the result of the cell below.
2.  Select **Visualization**.
3.  Choose **Histogram** to see the distribution of `age`.
4.  Choose **Box Plot** to see the distribution of `salary` (and spot outliers!).

In [0]:
display(df)

## Section 4: Identifying Outliers

**Why are outliers a problem?**
Outliers are extreme values that deviate significantly from other observations.
- **Impact on Mean:** A single billionaire in a neighborhood can skew the "average income" massively, making it unrepresentative.
- **Impact on Models:** Linear models (Regression) minimize "squared error". An outlier has a huge error, so the model will try too hard to fit it, ruining the fit for the rest of the data.

**The IQR Method:**
We use the **Interquartile Range (IQR)** because it is robust. Unlike Standard Deviation, it is not influenced by the outliers themselves.
Rule: A point is an outlier if it is above $Q3 + 1.5 \times IQR$.

In [0]:
# Calculate Q1 and Q3 for Salary
quantiles = df.approxQuantile("salary", [0.25, 0.75], 0.01)
q1, q3 = quantiles[0], quantiles[1]
iqr = q3 - q1

upper_bound = q3 + 1.5 * iqr

print(f"Q1: {q1}, Q3: {q3}, IQR: {iqr}")
print(f"Upper Bound for Outliers: {upper_bound}")

# Filter outliers
outliers = df.filter(df.salary > upper_bound)
print(f"Number of outliers found: {outliers.count()}")
display(outliers)

## Best Practices

### 🎯 EDA Strategy (in order of priority):

**1. Start with the big picture:**
- Check shape: `df.count()`, `len(df.columns)`
- Use Data Profile tab in Databricks
- Look at `df.printSchema()` for data types

**2. Identify quality issues:**
- Missing values: `df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])`
- Duplicates: `df.count() - df.dropDuplicates().count()`
- Invalid values: domain-specific checks (e.g., negative ages)

**3. Understand distributions:**
- Use `summary()` for numerical columns
- Check `skewness()` - values > 1 indicate strong skew
- Visualize with histograms and box plots

**4. Document findings:**
- Keep notes on data quality issues
- Document assumptions made
- Track decisions for handling outliers/missing values

| Metric | Good Value | Action if Exceeded |
|--------|------------|-------------------|
| Missing rate | < 5% | Consider imputation |
| Outlier rate | < 1% | Investigate and decide |
| Skewness | -1 to 1 | Consider log transform |
| Duplicate rate | 0% | Remove or investigate |

## Summary

### What we achieved:

- **Data Generation**: Created synthetic `customer_bronze` table with realistic defects
- **Data Profiling**: Used `summary()`, `skewness()`, and Data Profile tab
- **Visualizations**: Histograms and Box Plots for distribution analysis
- **Outlier Detection**: Applied IQR method to identify extreme values

### Key Takeaways:

| # | Principle |
|---|-----------|
| 1 | **Always start with EDA** - understand your data before modeling |
| 2 | **"Garbage In, Garbage Out"** - data quality determines model quality |
| 3 | **IQR is robust** - use it instead of standard deviation for outliers |
| 4 | **Skewness matters** - affects model choice and feature transformation |
| 5 | **Document everything** - track data quality issues and decisions |

### Metrics to Monitor:

| Metric | Our Data | Status |
|--------|----------|--------|
| Missing values (age) | ~5% | ⚠️ Needs imputation |
| Outliers (salary) | ~1% | ⚠️ Needs handling |
| Skewness (salary) | High | ⚠️ Consider log transform |

### Next Steps:

📚 **Next Module:** Module 2 - Data Splitting (train/test split strategies)

## Cleanup

Optionally remove demo tables created during exercises:

In [0]:
# Cleanup - remove demo tables created in this notebook

# Uncomment the lines below to remove demo tables:

# spark.sql(f"DROP TABLE IF EXISTS {catalog_name}.{schema_name}.customer_bronze")

# print("✅ All demo tables removed")

print("ℹ️ Cleanup disabled (uncomment code to remove demo tables)")